In [83]:
import os, io
from google.cloud import vision
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

In [ ]:
#add api file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'json api file name'


#add path to folder
FOLDER_PATH = r'path\to\folder'

#Get list of all screenshots in directory
FILE_LIST = os.listdir(FOLDER_PATH)
#add path to file
FILE_PATH = r'path\to\file'
#Initialise DF outside of for loop
raw_df = pd.DataFrame(columns=['locale','description'])
raw_df

In [ ]:
clean_df = pd.DataFrame(columns=['date',
                                'game_type',
                                'map',
                                'duration',
                                'result',
                                'player_name',
                                'avg_combat_score',
                                'k',
                                'd',
                                'a',
                                'econ_rating'])
clean_df

In [ ]:
input_file = FILE_LIST[0]
with io.open(os.path.join(FOLDER_PATH, input_file), 'rb') as image_file:
        content = image_file.read()
        
image = vision.Image(content=content)
response = client.text_detection(image=image)
document = response.full_text_annotation
#text = texts[0].description
#print(text)


In [ ]:
output_text = ''
for page in response.full_text_annotation.pages:
    for block in page.blocks:
        for paragraph in block.paragraphs:
            for word in paragraph.words:
                output_text += ''.join([
                    symbol.text for symbol in word.symbols
                ])
            output_text += '\n'
print(output_text)

In [ ]:
from enum import Enum

class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

def draw_boxes(input_file, bounds):
    img = cv2.imread(input_file, cv2.IMREAD_COLOR)
    for bound in bounds:
      p1 = (bound.vertices[0].x, bound.vertices[0].y) # top left
      p2 = (bound.vertices[1].x, bound.vertices[1].y) # top right
      p3 = (bound.vertices[2].x, bound.vertices[2].y) # bottom right
      p4 = (bound.vertices[3].x, bound.vertices[3].y) # bottom left
      cv2.line(img, p1, p2, (0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
      cv2.line(img, p2, p3, (0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
      cv2.line(img, p3, p4, (0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
      cv2.line(img, p4, p1, (0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
    return img

def get_document_bounds(response, feature):
    document = response.full_text_annotation
    bounds = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                          bounds.append(symbol.bounding_box)
                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)
                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)
            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)
    return bounds

In [ ]:

bounds = get_document_bounds(response, FeatureType.BLOCK)
img_block = draw_boxes(FILE_PATH, bounds)
#print(img_block)


bounds = get_document_bounds(response, FeatureType.PARA)
img_para = draw_boxes(FILE_PATH, bounds)
bounds = get_document_bounds(response, FeatureType.WORD)
img_word = draw_boxes(FILE_PATH, bounds)
bounds = get_document_bounds(response, FeatureType.SYMBOL)
img_symbol = draw_boxes(FILE_PATH, bounds)
#print(img_symbol)
#plt.imshow(img_block)

print("img_block")
plt.figure(figsize=[100,100]);plt.imshow(img_block[:,:,::-1]);plt.title("img_block")
print("img_para")

plt.figure(figsize=[100,100]);plt.imshow(img_para[:,:,::-1]);plt.title("img_para")
print("img_word")

plt.figure(figsize=[100,100]);plt.imshow(img_word[:,:,::-1]);plt.title("img_word")
print("img_symbol")

plt.figure(figsize=[100,100]);plt.imshow(img_symbol[:,:,::-1]);plt.title("img_symbol")